In [747]:
from math import ceil
import time, re, ast, sys, urllib, time, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle 
from header_list import user_agent_list
from proxies_list import proxies_list_, proxie_random_pool
from FUNCTIONS import proxie_check, strip_count, gen_url_list
from SCRAPPER_FUNCTIONS import session_creator, zip_prop_count, each_page, links_for_props
from MAIN_FUNCTIONS import address_parser, top_info_parser, public_info_parser, school_parser, feats_parser

In [739]:
ua = user_agent_list
proxies = proxies_list_
prx_pool = proxie_random_pool
# proxies = proxie_check(proxies)

In [740]:
len(proxies)

60

In [204]:
# zip_codes = pd.read_csv('../Data/main_zips.csv')
# zip_list = zip_codes['zip'].tolist()

In [264]:
# active_zip_url_list, sold_zip_url_list = gen_url_list(zip_list)

In [213]:
# a_main_df = pd.DataFrame(columns=['full_address', 'home_link'])
# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

In [261]:
# a = randint(0, 3)
# time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
# active_zip_url_list, a_main_df, proxies = links_for_props(proxies, active_zip_url_list, a_main_df, ua)

In [309]:
# a = randint(0, 3)
# sold_zip_url_list, s_main_df, proxies = links_for_props(proxies, sold_zip_url_list, s_main_df, ua)
# time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)

In [295]:
# a_main_df.to_csv('../Data/active_props_url_list.csv')
# s_main_df.to_csv('../Data/sold_props_url_list.csv')

In [652]:
# a_main_df = pd.read_csv('../Data/active_props_url_list.csv')
# s_main_df = pd.read_csv('../Data/sold_props_url_list.csv')

In [304]:
# a_main_df = a_main_df.drop_duplicates(subset='full_address')
# s_main_df = s_main_df.drop_duplicates(subset='full_address')

In [305]:
active_url_list = a_main_df.home_link.tolist()
sold_url_list = s_main_df.home_link.tolist()

In [763]:
def each_property(url, hdr, proxy):

    soup = session_creator(proxy, ua, url)

    home_dict = address_parser(soup)

    top_info_dict = top_info_parser(soup)

    public_info_dict = public_info_parser(soup)

    school_dict = school_parser(soup)

    all_home_feats = feats_parser(soup)
    
    df = pd.DataFrame()
    
    df = pd.concat([home_dict, top_info_dict, top_info_dict, public_info_dict, all_home_feats], axis=1)

    return df

In [765]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
sold_url_list = s_main_df['home_link'].tolist()
# url = random.sample(active_url_list, 1)[0]
url = random.sample(sold_url_list, 1)[0]
print(proxy)
print(url)

sold_home_df = each_property(url, header, proxy)

69.38.69.254:52168
https://www.redfin.com/CA/Santa-Rosa/2051-Gardenview-Pl-95403/home/2640288


In [766]:
sold_home_df

,address,city,zip_code,state,description,mls_num,num_beds,num_baths,home_sqft,yr_blt,...,Property Subtype 1:,Style:,Age:,Year Built:,Year Built Source:,Square Footage:,Planned Unit Develop:,Property Disclaimer:,Original Price:,Sale/Lease-Rent:
1,2051 Gardenview Pl,Santa Rosa,95403,None,Enjoy Sonoma County living in this beautifully...,21818180,4.0,3.0,872.0,2000,...,Single Family Residence,Unknown,0,2000,Realist Public Rec,"2,872",Yes,"Information has not been verified, is not guar...",995000.00,Sale
